In [1]:
import cv2                 
import numpy as np         
import os 

In [2]:
# CTE: These are my filepaths, here for convenience
cte_train_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\train\\'
cte_test_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\test\\'
cte_additional_t1_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_1_v2\\'
cte_additional_t2_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_2_v2\\'
cte_additional_t3_dir = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\additional_Type_3_v2\\'

TRAIN_DIR = cte_train_dir # put the path to the training data here.
TEST_DIR = cte_test_dir # put the path to the test data here.
IMG_SIZE = 50 # we can change this to whatever
STORAGE_DIR = 'D:\\Work\\Kaggle\\Cancer Screening\\Cancer-Screening-NN\\Data\\processed\\'

In [3]:
# Progress bar function, since some of these processes take a while
def show_progress(current_index, total_len):
    x = (current_index + 1) * 100 // total_len
    x_less1 = current_index * 100 // total_len
    if (x % 10 == 0 and x - x_less1 > 0 and x < 100):
        print('{}%...'.format(x), end='')
    elif current_index == total_len - 1:
        print('100%')

In [4]:
def label_img(file):
    data_label = file
    # conversion to one-hot array [type_1,type_2,type_3]
    if data_label == 'Type_1': return [1,0,0]
    elif data_label == 'Type_2': return [0,1,0]
    elif data_label == 'Type_3': return [0,0,1]

In [5]:
def create_train_data(img_size, train_dir):
    training_data = []
    for file in os.listdir(train_dir):
        print('Starting on file {}'.format(file))
        file_path = os.path.join(train_dir,file)
        data_label = label_img(file)
        file_len = len(os.listdir(file_path))
        for i, img in enumerate(os.listdir(file_path)):
            show_progress(i,file_len) # Print progress
            label = data_label
            path = os.path.join(file_path,img)
            try:
                img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (img_size,img_size))
            except:
                print('\nError resizing image {} in {}'.format(img, file_path))
                continue
            training_data =(np.array(img),np.array(label))
            # So that we don't have to do this again, unless we change something here.
            np.save('{}{}\\{}.npy'.format(STORAGE_DIR, label_folder, len(os.listdir(STORAGE_DIR+label_folder))), training_data)
#    return training_data

In [6]:
def process_test_data():
    testing_data = []
    file_len = len(os.listdir(TEST_DIR))
    for i, img in enumerate(os.listdir(TEST_DIR)):
        show_progress(i,file_len) # Print progress
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

In [7]:
train_folders = [cte_train_dir, cte_additional_t1_dir, cte_additional_t2_dir, cte_additional_t3_dir]
for folder in train_folders:
    train_data = create_train_data(IMG_SIZE, folder)
#process_test_data()

Starting on file Type_1
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
Starting on file Type_2
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
Starting on file Type_3
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
Starting on file Type_1
10%...20%...30%...40%...50%...60%...70%...
Error resizing image 5893.jpg in D:\Work\Kaggle\Cancer Screening\Cancer-Screening-NN\Data\additional_Type_1_v2\Type_1
80%...90%...100%
Starting on file Type_2
10%...20%...30%...
Error resizing image 2845.jpg in D:\Work\Kaggle\Cancer Screening\Cancer-Screening-NN\Data\additional_Type_2_v2\Type_2
40%...50%...60%...70%...
Error resizing image 5892.jpg in D:\Work\Kaggle\Cancer Screening\Cancer-Screening-NN\Data\additional_Type_2_v2\Type_2
80%...90%...100%
Starting on file Type_3
10%...20%...30%...40%...50%...60%...70%...80%...90%...100%
